# 파이썬 파이널 과제 풀이

In [2]:
# 데이터셋 ≒ 데이터 마트 ≒ 테이블 개념
    # 데이터분석가 신입 주니어로 입사하면: 
        # 데이터셋 구축하는 업무 주로 받음
# 데이터 웨어하우스 ≒ 데이터베이스 개념

## 01. 패키지 참조

In [9]:
import requests
from pandas import DataFrame
from concurrent import futures

## 02. CSV 파일 읽어오기

In [14]:
with open("서울교통공사_역주소 및 전화번호_20250318 (1).csv", "r", encoding="euc-kr") as f:
    csv_list = f.readlines()

print(csv_list[:5])

['연번,역번호,호선,역명,역전화번호,도로명주소,지번주소\n', '1,150,1,서울,02-6110-1331,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선)\n', '2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선)\n', '3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선)\n', '4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선)\n']


## 03. OpenAPI data request 스펙 확인

### 요청 정보 확인 (확인용)

In [40]:
# 요청 URL
url = "https://api.vworld.kr/req/address"

# QueryString 요청변수
params = {
            "service": "address",
            "request": "GetCoord",
            "crs": "epsg:4326",
            "address": "판교로 242",
            "format": "json",
            "type": "ROAD",
            "key": "A9FEA51E-B101-3762-AE06-592EED1EE5DE"
        }

### 웹 데이터 요청하기

In [41]:
with requests.Session() as session:
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    })

    r = session.get(url, params=params)

    if r.status_code != 200:
        msg = "[%d error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
        
    print(r)


<Response [200]>


### 결과 데이터 확인

In [42]:
mydict = r.json()
mydict

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'GetCoord',
   'time': '18(ms)'},
  'status': 'OK',
  'input': {'type': 'ROAD', 'address': '판교로 242'},
  'refined': {'text': '경기도 성남시 분당구 판교로 242 (삼평동)',
   'structure': {'level0': '대한민국',
    'level1': '경기도',
    'level2': '성남시 분당구',
    'level3': '삼평동',
    'level4L': '판교로',
    'level4LC': '',
    'level4A': '삼평동',
    'level4AC': '4113565500',
    'level5': '242',
    'detail': ''}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '127.101313354', 'y': '37.402352535'}}}}

### 위경도 추출

In [43]:
point = mydict['response']['result']["point"]
print(point['x'], point['y'])

127.101313354 37.402352535


## 04. 위경도 조회 함수 짜기

In [59]:
def get_point(addr, type="road"): #기본값=road (비동기 처리할 때 type 변환 있을거라 변수로 지정한거임)
    #요청 url
    url = "https://api.vworld.kr/req/address"

    # QueryString 요청변수
    params = {
        "service": "address",
        "request": "GetCoord",
        "crs": "epsg:4326",
        "address": addr,
        "format": "json",
        "type": type, #비동기 처리할 때 type 변환 있을거라 변수로 지정한거임
        "key": "A9FEA51E-B101-3762-AE06-592EED1EE5DE"
    }

    with requests.Session() as session:
        session.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        })

        r = session.get(url, params=params)

        if r.status_code != 200:
            msg = "[%d error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)

    mydict = r.json()
    point = mydict['response']['result']["point"]
    return (point['x'], point['y'])
    

## 05. 위경도 변환하기

### 1) 동기식 처리 ver.

In [60]:
size = len(csv_list)
resultset = []      # 결과 저장용 빈 리스트

# 약식으로 10건만 진행 (걸리는 시간만 보려고, 시간이슈)
# for i, v in enumerate(csv_list[1:]):
for i, v in enumerate(csv_list[1:11]):
    print("%d/%d 진행중..." % (i+1, size))
    items = v.strip().split(",")

    try:
        lat, lon = get_point(items[5])
    except Exception as e:
        # 도로명 주소로 실패했다면 지번 주소로 츄라이
        try:
            lat, lon = get_point(items[6], type="parcel")
        except Exception as e2:
            lat = None
            lon = None
    
    items.append(lat)
    items.append(lon)
    resultset.append(items)

resultset

1/290 진행중...
2/290 진행중...
3/290 진행중...
4/290 진행중...
5/290 진행중...
6/290 진행중...
7/290 진행중...
8/290 진행중...
9/290 진행중...
10/290 진행중...


[['1',
  '150',
  '1',
  '서울',
  '02-6110-1331',
  '서울특별시 중구 세종대로 지하2(남대문로 5가)',
  '서울특별시 중구 남대문로5가 73-6 서울역(1호선)',
  '126.97254323694754',
  '37.55702928798156'],
 ['2',
  '151',
  '1',
  '시청',
  '02-6110-1321',
  '서울특별시 중구 세종대로 지하101(정동)',
  '서울특별시 중구 정동 5-5 시청역(1호선)',
  '126.978346780',
  '37.566700969'],
 ['3',
  '152',
  '1',
  '종각',
  '02-6110-1311',
  '서울특별시 종로구 종로 지하55(종로1가)',
  '서울특별시 종로구 종로1가 54 종각역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['4',
  '153',
  '1',
  '종로3가',
  '02-6110-1301',
  '서울특별시 종로구 종로 지하129(종로3가)',
  '서울특별시 종로구 종로3가 10-5 종로3가역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['5',
  '154',
  '1',
  '종로5가',
  '02-6110-1291',
  '서울특별시 종로구 종로 지하216(종로5가)',
  '서울특별시 종로구 종로5가 82-1 종로5가역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['6',
  '155',
  '1',
  '동대문',
  '02-6110-1281',
  '서울특별시 종로구 종로 지하302(창신동)',
  '서울특별시 종로구 창신동 492-1 동대문역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['7',
  '156',
  '1',
  '신설동',
  '02-6110-1261',
  '서울특별시 동대문구 왕산로 지하1(신설동)',
 

### 2) 비동기 처리 ver

In [ ]:
size = len(csv_list)
processes = []      # 비동기 과정을 저장할 리스트
resultset = []      # 비동기 작업결과를 저장할 리스트

with futures.ThreadPoolExecutor(max_workers=30) as executor:
    for i, v in enumerate(csv_list[1:]):
        print("%d/%d 진행중..." % (i+1, size))
        items = v.strip().split(",")

        pro = executor.submit(get_point, items[5])  # 비동기 작업 생성
        processes.append(pro)
    
    for i, p in enumerate(processes):
        try:
            # try~except: 비동기 처리가 실행되는 동안 발생하는 예외 대비문
            lat, lon = p.result()
        except Exception as e:
            try:
                lat, lon = get_point(items[6], type="parcel")
            except Exception as e2:
                lat = None
                lon = None

        # 새로운 반복문이므로 직전 반복문에서 생성한 items 변수가 유효하지 않음
        # -> items 변수를 새로 생성해야함
        items = csv_list[i].strip().split(",")
        items.append(lat)
        items.append(lon)
        resultset.append(items)

resultset

1/290 진행중...
2/290 진행중...
3/290 진행중...
4/290 진행중...
5/290 진행중...
6/290 진행중...
7/290 진행중...
8/290 진행중...
9/290 진행중...
10/290 진행중...
11/290 진행중...
12/290 진행중...
13/290 진행중...
14/290 진행중...
15/290 진행중...
16/290 진행중...
17/290 진행중...
18/290 진행중...
19/290 진행중...
20/290 진행중...
21/290 진행중...
22/290 진행중...
23/290 진행중...
24/290 진행중...
25/290 진행중...
26/290 진행중...
27/290 진행중...
28/290 진행중...
29/290 진행중...
30/290 진행중...
31/290 진행중...
32/290 진행중...
33/290 진행중...
34/290 진행중...
35/290 진행중...
36/290 진행중...
37/290 진행중...
38/290 진행중...
39/290 진행중...
40/290 진행중...
41/290 진행중...
42/290 진행중...
43/290 진행중...
44/290 진행중...
45/290 진행중...
46/290 진행중...
47/290 진행중...
48/290 진행중...
49/290 진행중...
50/290 진행중...
51/290 진행중...
52/290 진행중...
53/290 진행중...
54/290 진행중...
55/290 진행중...
56/290 진행중...
57/290 진행중...
58/290 진행중...
59/290 진행중...
60/290 진행중...
61/290 진행중...
62/290 진행중...
63/290 진행중...
64/290 진행중...
65/290 진행중...
66/290 진행중...
67/290 진행중...
68/290 진행중...
69/290 진행중...
70/290 진행중...
71/290 진행중...
72/290 진행중...
7

[['연번',
  '역번호',
  '호선',
  '역명',
  '역전화번호',
  '도로명주소',
  '지번주소',
  '127.14834880144522',
  '37.528334424738375'],
 ['1',
  '150',
  '1',
  '서울',
  '02-6110-1331',
  '서울특별시 중구 세종대로 지하2(남대문로 5가)',
  '서울특별시 중구 남대문로5가 73-6 서울역(1호선)',
  '126.978346780',
  '37.566700969'],
 ['2',
  '151',
  '1',
  '시청',
  '02-6110-1321',
  '서울특별시 중구 세종대로 지하101(정동)',
  '서울특별시 중구 정동 5-5 시청역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['3',
  '152',
  '1',
  '종각',
  '02-6110-1311',
  '서울특별시 종로구 종로 지하55(종로1가)',
  '서울특별시 종로구 종로1가 54 종각역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['4',
  '153',
  '1',
  '종로3가',
  '02-6110-1301',
  '서울특별시 종로구 종로 지하129(종로3가)',
  '서울특별시 종로구 종로3가 10-5 종로3가역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['5',
  '154',
  '1',
  '종로5가',
  '02-6110-1291',
  '서울특별시 종로구 종로 지하216(종로5가)',
  '서울특별시 종로구 종로5가 82-1 종로5가역(1호선)',
  '127.022379505',
  '37.573411520'],
 ['6',
  '155',
  '1',
  '동대문',
  '02-6110-1281',
  '서울특별시 종로구 종로 지하302(창신동)',
  '서울특별시 종로구 창신동 492-1 동대문역(1호선)',
  '127.0250937

## 05. 변환 결과 저장하기

### 데이터프레임으로 변환

In [56]:
new_resultset = []

for r in resultset:
    item_dict = {
        "연변": r[0],
        "역번호": r[1],
        "호선": r[2],
        "역명": r[3],
        "역전화번호": r[4],
        "도로명주소": r[5],
        "지번주소": r[6],
        "위도": r[7],
        "경도": r[8]
    }

    new_resultset.append(item_dict)

df = DataFrame(new_resultset)
df.to_excel("지하철역.xlsx")
df

,연변,역번호,호선,역명,역전화번호,도로명주소,지번주소,위도,경도
0,연번,역번호,호선,역명,역전화번호,도로명주소,지번주소,127.14834880144522,37.528334424738375
1,1,150,1,서울,02-6110-1331,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선),126.978346780,37.566700969
2,2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선),127.022379505,37.573411520
3,3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선),127.022379505,37.573411520
4,4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선),127.022379505,37.573411520
...,...,...,...,...,...,...,...,...,...
284,284,4133,9,석촌,02-2656-0933,서울특별시 송파구 송파대로 지하439(석촌동),서울특별시 송파구 석촌동 209 석촌역(9호선),127.101061494,37.502815623
285,285,4134,9,송파나루,02-2656-0934,서울특별시 송파구 백제고분로 지하446(방이동),서울특별시 송파구 방이동 2 송파나루역(9호선),127.118345402,37.512011982
286,286,4135,9,한성백제,02-2656-0935,서울특별시 송파구 위례성대로 지하29(방이동),서울특별시 송파구 방이동 88-17 한성백제역(9호선),127.134326509,37.515450096
287,287,4136,9,올림픽공원(한국체대),02-2656-0936,서울특별시 송파구 양재대로 지하1233(방이동),서울특별시 송파구 방이동 89-28 올림픽공원역(9호선),127.132180703,37.524208018


### 변환결과 엑셀로 저장

In [58]:
df.to_excel("지하철역_위치.xlsx")